In [2]:
import numpy as np # library to handle data in a vectorized manner
import time
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [8]:
# Shenton Way, District 01, Singapore
address = '16545 South West, Miami'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Miami home are {}, {}.'.format(latitude, longitude))

C:\Users\Juan Raul Garcia\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of Miami home are 43.0315001, -76.1541515.


In [9]:
neighborhood_latitude=43.0315001
neighborhood_longitude=-76.1541515

In [10]:
CLIENT_ID = '54GU41KXZPLVE2IUFD2TSPOJVTLCROC1VTWAEYK3YEKIBYU5' # your Foursquare ID
CLIENT_SECRET = 'P2DVHDL0WHSWNSZHSWKJBEBZXRROKWDO2APFCQWQ1CJMFJMG' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 54GU41KXZPLVE2IUFD2TSPOJVTLCROC1VTWAEYK3YEKIBYU5
CLIENT_SECRET:P2DVHDL0WHSWNSZHSWKJBEBZXRROKWDO2APFCQWQ1CJMFJMG


In [11]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=54GU41KXZPLVE2IUFD2TSPOJVTLCROC1VTWAEYK3YEKIBYU5&client_secret=P2DVHDL0WHSWNSZHSWKJBEBZXRROKWDO2APFCQWQ1CJMFJMG&v=20180604&ll=43.0315001,-76.1541515&radius=500&limit=100'

In [12]:
# results display is hidden for report simplification 
results = requests.get(url).json()
#results

In [13]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [14]:
venues = results['response']['groups'][0]['items']
SGnearby_venues = json_normalize(venues) # flatten JSON
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
SGnearby_venues =SGnearby_venues.loc[:, filtered_columns]
# filter the category for each row
SGnearby_venues['venue.categories'] = SGnearby_venues.apply(get_category_type, axis=1)
# clean columns
SGnearby_venues.columns = [col.split(".")[-1] for col in SGnearby_venues.columns]

SGnearby_venues.shape

(5, 4)

In [15]:
# Venues near current Singapore residence place
SGnearby_venues.head()

,name,categories,lat,lng
0,Cuse Dowgs Unleashed,Bar,43.033076,-76.155835
1,H.D.M. Paving and Sealing,Construction & Landscaping,43.028480,-76.152800
2,Reis Beach,Portuguese Restaurant,43.030766,-76.159182
3,Jamaica Lick Finger Restaurant,Food,43.033270,-76.158888
4,KFC,Fried Chicken Joint,43.031114,-76.148291


In [16]:
latitude=43.0315001
longitude=-76.1541515
# create map of Singapore place  using latitude and longitude values
map_sg = folium.Map(location=[latitude, longitude], zoom_start=18)

In [18]:
#add markers to map
for lat, lng, label in zip(SGnearby_venues['lat'], SGnearby_venues['lng'], SGnearby_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.RegularPolygonMarker(
        [lat, lng],
        number_of_sides=30,
        radius=7,
        popup=label,
        color='blue',
        fill_color='#0f0f0f',
        fill_opacity=0.6,
    ).add_to(map_sg)  
    
map_sg